<a href="https://colab.research.google.com/github/DSbyte/recommender-system/blob/main/Amazon_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Recommendation System: Amazon Products




In [1]:
from collections import Counter
from google.colab import drive
from torch import nn, optim
from tqdm.auto import tqdm
import torch, torchtext, numpy as np
import pandas as pd, csv
import matplotlib.pyplot as plt
import pdb

torch.manual_seed(291)
np.random.seed(291)

In [2]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
#Reading data
df = pd.read_csv('/content/gdrive/MyDrive/ML/ratings_Electronics (1).csv')

In [4]:
df

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600
...,...,...,...,...
7824476,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824477,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


In [5]:
#Renaming dataframe columns to make the data easier to work with
df = df.rename(columns={"AKM1MP6P0OYPR":"userId","0132793040":"productId","5.0":"rating","1365811200":"timeStamp"})

In [8]:
df.head()

,userId,productId,rating,timeStamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [9]:
type(df.head(1))

pandas.core.frame.DataFrame

In [10]:
len(df['productId'].unique())

476001

Hashing to combat alpha numeric values that are not accepted by pytorch long tensor


In [ ]:
#trash = {u: n for n,u in enumerate(df['userId'].unique())}

In [ ]:
# map = []

# map.append(1)
# map.append((2,3))
# mapped = {}


In [ ]:
# type(df['userId'].tolist())

In [ ]:


# def userIdtoNum(user_idslist):
#   temp = 0
#   user_ids = user_idslist.tolist()
#   count = 0
#   list_count = []
#   list_users = []
#   mapped = []
#   # print(mapped)

#   # for i in mapped:
#   #   print(i[0], i[1])

#   for i in range(len(user_ids)):
#     if i==0:
#       list_count.append(count)
#       list_users.append(user_ids[i])
#       # print(list_count)
#       # print(list_users)
#       # print(mapped)
#       prev = user_ids[i]


#     else:
#       curr = []
#       curr.append(user_ids[i])
#       counter_available = Counter(list_users)
#       counter_needed = Counter(curr)
#       out = counter_needed - counter_available
#       if not list(out.elements()):
#         count += 1
#         list_count.append(count)
#         # print(list_count)
#         list_users.append(curr)
#         # print(list_users)
#       else:
#         temp += 1
#         list_count.append(count)
#         list_users.append(curr)
      
#       # curr = user_ids[i]

#       # for cnt,users in list(zip(list_count,list_users)):
#       #   if users == curr:
#       #     temp += 1
#       #     list_count.append(cnt)
#       #     list_users.append(curr)
#       #     break
#       # else:
#       #   count += 1
#       #   list_count.append(count)
#       #   # print(list_count)
#       #   list_users.append(curr)
#       #   # print(list_users)
#       prev = curr

#   mapped = list(zip(list_count,list_users))
#   print(temp)
#   return mapped
#   pass



In [6]:
q = []
p = df['userId'].tolist()
for i in p:
  q.append(hash(i))
  
df_col = pd.DataFrame({'userHashed':q})

df['userId'] = df_col['userHashed']

In [7]:
r = []
s = df['productId'].tolist()
for i in s:
   r.append(hash(i))
  
df_col2 = pd.DataFrame({'productHashed':r})
df['productId'] = df_col2['productHashed']

In [13]:
print(df.head())

                userId            productId  rating   timeStamp
0  3094524065642534589  7228691163309702983     5.0  1341100800
1 -7357476511047441823  4203638183892874418     1.0  1367193600
2  3255321322911387393  4203638183892874418     3.0  1374451200
3  2181845487306584024  4203638183892874418     1.0  1334707200
4  4510243544807772982 -6327205876201498854     5.0  1397433600


In [ ]:
# print(df.head())

0    2928384221098543557
1    3967222656333255743
2    3967222656333255743
3    3967222656333255743
4    -233870297543730163
Name: productId, dtype: int64


In [ ]:
# df_100k = df.sample(n=100000)


In [ ]:
# torch.LongTensor((df_100k[['userId', 'productId']].values))

tensor([[ 3745394179866338730,  4554702336353503915],
        [ 7648718285032043986, -5293317672808570896],
        [ 2407203665604160630, -6284330153785957090],
        ...,
        [-1354225317536763324, -1542084943365013846],
        [ -989104399912570678,  4198949797512024165],
        [ 2534438094971252782, -4989503219565073604]])

In [ ]:
# df_100k['userId']

5033467    3745394179866338730
323687     7648718285032043986
4527720    2407203665604160630
4571079    7074620996651281036
4325424   -8427743009652482748
                  ...         
2991321    7004361719798029246
3247997    6061416206100637638
7521897   -1354225317536763324
832255     -989104399912570678
1151105    2534438094971252782
Name: userId, Length: 100000, dtype: int64

In [8]:
class ProductDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        u2n = { u: n for n, u in enumerate(df['userId'].unique()) }
        p2n = { p: n for n, p in enumerate(df['productId'].unique()) }
        df['userId'] = df['userId'].apply(lambda u: u2n[u])
        df['productId'] = df['productId'].apply(lambda p: p2n[p])
        self.coords = torch.LongTensor(df[['userId','productId']].values)
        self.ratings = torch.FloatTensor(df['rating'].values)
        self.n_users = df['userId'].nunique()
        self.n_products = df['productId'].nunique()

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, i):
        return (self.coords[i], self.ratings[i])

In [ ]:
#Creating train and test datasets
# ds_full = ProductDataset(df_100k)

# n_train = int(0.8 * len(ds_full))
# n_test = len(ds_full) - n_train
# rng = torch.Generator().manual_seed(291)
# ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

In [9]:
class ProductRecommendation(nn.Module):
    def __init__(self, n_users, n_products, emb_dim):
        super(ProductRecommendation, self).__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.user_bias = nn.Embedding(n_users, 1)
        self.product_emb = nn.Embedding(n_products, emb_dim)
        self.product_bias = nn.Embedding(n_products, 1)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.product_emb.weight)
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.product_bias.weight)
    
    def forward(self, samples):
        users = self.user_emb(samples[:,0])
        products = self.product_emb(samples[:,1])
        dot = (users * products).sum(1)
        user_b = self.user_bias(samples[:,0]).squeeze()
        product_b = self.product_bias(samples[:,1]).squeeze()
        return torch.sigmoid((dot + user_b + product_b))*5.0

In [10]:
device = torch.device('cuda')

def run_test(model, ldr, crit):
    total_loss, total_count = 0, 0
    model.eval()
    tq_iters = tqdm(ldr, leave=False, desc='test iter')
    with torch.no_grad():
        for coords, labels in tq_iters:
            coords, labels = coords.to(device), labels.to(device)
            preds = model(coords)
            loss = crit(preds, labels)
            total_loss += loss.item() * labels.size(0)
            total_count += labels.size(0)
            tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_train(model, ldr, crit, opt, sched):
    model.train()
    total_loss, total_count = 0, 0
    tq_iters = tqdm(ldr, leave=False, desc='train iter')
    for (coords, labels) in tq_iters:
        opt.zero_grad()
        coords, labels = coords.to(device), labels.to(device)
        preds = model(coords)
        loss = crit(preds, labels)
        loss.backward()
        opt.step()
        sched.step()
        total_loss += loss.item() * labels.size(0)
        total_count += labels.size(0)
        tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs=10):
    best_loss = np.inf
    tq_epochs = tqdm(range(n_epochs), desc='epochs', unit='ep')
    for epoch in tq_epochs:
        train_loss = run_train(model, ldr_train, crit, opt, sched)
        test_loss = run_test(model, ldr_test, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f}    test loss {test_loss:.6f}')
        if test_loss < best_loss:
            best_loss = test_loss
            tq_epochs.set_postfix({'bE': epoch, 'bL': best_loss}, refresh=True)

In [12]:

# df_500k = df


ds_full = ProductDataset(df)

n_train = int(0.8 * len(ds_full))
n_test = len(ds_full) - n_train
rng = torch.Generator().manual_seed(291)
ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

model = ProductRecommendation(ds_full.n_users, ds_full.n_products, 20)
model.to(device)


ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

n_epochs = 5000

crit = nn.MSELoss().to(device)
opt = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs=n_epochs)

run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs)

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), "/content/gdrive/MyDrive/ML/params")

In [ ]:
# !pip install torchensemble
# from torchensemble import BaggingClassifier
# from torchensemble.utils.logging import set_logger
# device = torch.device('cpu')

# model_ensemble = BaggingClassifier(
#     estimator=model,
#     n_estimators=10,
#     cuda=False,
# )
# model_ensemble.to(device)
# model_ensemble.set_optimizer('SGD',
#                               lr=1e-6,
#                               momentum=0.9
# )
# model_ensemble.fit(train_loader=ldr_train, epochs=5, test_loader=ldr_test) 
# accuracy = model_ensemble.predict(test_loader=ldr_test)

In [ ]:
# model_ensemble.to(device)

# ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
# ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

# n_epochs = 5

# crit = nn.MSELoss().to(device)
# opt = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)
# sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs=n_epochs)

# run_all(model_ensemble.fit(ldr_train,epochs=5), ldr_train, ldr_test, crit, opt, sched, n_epochs)
# model_ensemble.fit(ldr_train,epochs=5)
# opt = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)
# sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs=n_epochs)
# crit = nn.MSELoss().to(device)
# run_all(model_ensemble,ldr_train,ldr_test,crit,opt,sched,n_epochs=5)

In [ ]:
# torch.save(model.state_dict(), "/content/gdrive/MyDrive/ML/params")

In [ ]:
# df_10 = df.sample(n=10)
# df_10

,userId,productId,rating,timeStamp
7509855,983973915694958604,-6118496346122769748,5.0,1396051200
5433771,-8200340582553172797,-2426004328992617287,5.0,1341273600
2970913,-4087721239828602131,-3363225249964807121,5.0,1270598400
6585150,-5828237825798718144,-5154498351802633617,5.0,1388275200
1459402,4436744305774423256,3567849539807120695,5.0,1362960000
966473,2278172338605401606,-7311908809851851840,3.0,1387584000
974585,6234707495349648,5729682734443307242,5.0,1276300800
6856791,-2014509671718176939,966491560468669416,5.0,1393286400
7415058,-5640414658642003187,-7198301951599166732,5.0,1382659200
6401116,-2478918726768812794,-2657286959899449048,5.0,1376870400


In [ ]:
# df_5 = df_10.sample(n=5)
# df_5

,userId,productId,rating,timeStamp
7415058,-5640414658642003187,-7198301951599166732,5.0,1382659200
1459402,4436744305774423256,3567849539807120695,5.0,1362960000
6585150,-5828237825798718144,-5154498351802633617,5.0,1388275200
966473,2278172338605401606,-7311908809851851840,3.0,1387584000
6401116,-2478918726768812794,-2657286959899449048,5.0,1376870400


In [ ]:
# df_new = df_10.drop(df_5.index)
# df_new

,userId,productId,rating,timeStamp
7509855,983973915694958604,-6118496346122769748,5.0,1396051200
5433771,-8200340582553172797,-2426004328992617287,5.0,1341273600
2970913,-4087721239828602131,-3363225249964807121,5.0,1270598400
974585,6234707495349648,5729682734443307242,5.0,1276300800
6856791,-2014509671718176939,966491560468669416,5.0,1393286400
